Lets implement the sub question 2, the goal is to find a relation btw the number of views and the titles (length, sentiment, casefold, etc...). For that an observational studies would be highly recommended to compare what is comparable. For that we will keep the channel id to see the number of subs of the channel and compare pairwise similar nb of subs. 


In [ ]:
import pandas as pd
import re
from statsmodels.distributions.empirical_distribution import ECDF
import statsmodels.formula.api as smf 
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
video_metadatas = pd.read_feather("../data/yt_metadata_title_filter_cat_sample.feather")
df_time_series = pd.read_csv(
    "../data/df_timeseries_en.tsv.gz", compression="infer", sep="\t"
)
video_metadatas = video_metadatas.rename({"index": "id"}, axis=1)


In [ ]:
video_metadatas.head(5)


In [ ]:
df_time_series.head(5)


In [ ]:
# we need to improve the hw1 method a bit because we want to make sure that titles with a lot of spaces
# don't have too many words and if there is a typo (e.g "Welcome,in my video" should be counted as 4 words and not 3)
def count_words(x: str):
    new_x = x.replace(",", " ")
    # change anything that isn’t an alphanumeric character or whitespace, and replaces it with a space
    new_x = re.sub(r"[^\w\s]", " ", new_x)
    # Change many consecutive spaces into a single space
    new_x = re.sub(" +", " ", new_x)
    # delete begin/end spaces
    new_x = new_x.strip()
    return len(new_x.split(" "))


In [ ]:
print(count_words("Salut     je suis la"))
print(count_words("Salut,je suis la"))
print(count_words("#FuckCancer | I'VE GOT SOME GREAT NEWS!"))
print(count_words("DISNEY CHRISTMAS VLOG! || Zak Longo"))
print(count_words("Sims 4 - SHOOTING SIMS WITH A GUN - The Sims 4"))
print(count_words("###ADA#ada##"))


In [ ]:
video_metadatas["nb_words"] = video_metadatas.title.apply(
    lambda title: count_words(title)
)


In [ ]:
def get_freq_capital_words(sentence: str):
    new_x = sentence.replace(",", " ")
    # change anything that isn’t an alphanumeric character or whitespace, and replaces it with a space
    new_x = re.sub(r"[^\w\s]", " ", new_x)
    # Change many consecutive spaces into a single space
    new_x = re.sub(" +", " ", new_x)
    # delete begin/end spaces
    new_x = new_x.strip().split(" ")
    nb_capital_words = 0
    for word in new_x:
        if word.isupper():
            nb_capital_words += 1
    return nb_capital_words / len(new_x)


print(get_freq_capital_words("Sims 4 - SHOOTING SIMS WITH A GUN - The Sims 4"))
print(get_freq_capital_words("###ADA###salut|||test&SALUT"))


In [ ]:
video_metadatas["freq_capitalize_words"] = video_metadatas.title.apply(
    lambda title: get_freq_capital_words(title)
)


### Analyze the impact of the upper word in the titles.  


In [ ]:
# distrib of the freq capitalize words.
# process for the plot
freq_capit_words = round(video_metadatas.freq_capitalize_words, 1)
values_counts = freq_capit_words.value_counts().sort_index()
values_counts.plot(kind="bar")
plt.xlabel("frequencies of upper word")
plt.ylabel("nb of titles")


In [ ]:
video_metadatas["view_count_log"] = np.log(video_metadatas.view_count, where=video_metadatas.view_count != 0)
mod = smf.ols(formula="view_count_log ~ nb_words + freq_capitalize_words", data=video_metadatas)
res = mod.fit()
print(res.summary())

First results, it seems that the upper words seem to impact the number of views for videos. 

Map channels with same nb of followers, but videos with 1 in freq and 0 on the other side and compare the nb of views. 

In [ ]:
df_time_series.sample(3)

In [ ]:
# take df with only 1 and 0 in frequency 
video_metadatas.sample(2)

In [ ]:
binary_freq_df = video_metadatas[
    (video_metadatas.freq_capitalize_words == 0)
    | (video_metadatas.freq_capitalize_words == 1)
]
binary_freq_df["upload_date"] = pd.to_datetime(binary_freq_df["upload_date"])

In [ ]:
binary_freq_df.sample(5)

In [ ]:
no_upper_word_df = binary_freq_df[binary_freq_df["freq_capitalize_words"] == 0]
full_upper_word_df = binary_freq_df[binary_freq_df["freq_capitalize_words"] == 1]

In [ ]:
already_matched = {}
df_list = []
# match same year videos, same category and equal nb of words
i = 0
shuffle_full_upper_word_df = full_upper_word_df.sample(frac=1, random_state=1)
shuffle_no_upper_word_df = no_upper_word_df.sample(frac=1, random_state=1)

for _, full_upper_row in shuffle_full_upper_word_df.iterrows():
    if i == 100:
        print("We have 100 pairs")
        break
    for _, no_upper_row in shuffle_no_upper_word_df.iterrows(): 
        if (
            (full_upper_row.categories == no_upper_row.categories)
            and (full_upper_row.nb_words == no_upper_row.nb_words)
            and (full_upper_row.upload_date.year == no_upper_row.upload_date.year)
            # and (no_upper_row.id not in already_matched.values())
        ):
            already_matched[full_upper_row.id] = no_upper_row.id
            df_list.append(full_upper_row)
            df_list.append(no_upper_row)
            i += 1
            print("{}th match ".format(i))
            shuffle_no_upper_word_df.drop(shuffle_no_upper_word_df[shuffle_no_upper_word_df.id == no_upper_row.id].index, inplace=True)
            # print(shuffle_no_upper_word_df.shape)
            break


matched_df = pd.DataFrame(df_list)

In [ ]:
matched_df.head(4)

In [ ]:
mod = smf.ols(formula="view_count_log ~ freq_capitalize_words", data=matched_df)
res = mod.fit()
print(res.summary())